In [42]:
#Author: Henry Moore, with help from Divik Verma and Catherine Chu
#Date: July 24, 2024
#Assignment: Math 76.01 Homework 4, Decision trees, interpretability, and algorithmic bias

import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC

#1.1
# load data
raw_data = pd.read_csv('/Users/henrymoore/Desktop/mathai2/compas-scores-two-years.csv')
# print a list of variable names
print(raw_data.columns)
# look at the first 5 rows 
raw_data.head(5)

#1.2
# Select features and response variables

# Features by type
numerical_features = ['juv_misd_count', 'juv_other_count', 'juv_fel_count', 
    'priors_count', 'age']
binary_categorical_features = ['sex', 'c_charge_degree']
other_categorical_features = ['race']
all_features = binary_categorical_features + other_categorical_features + numerical_features

# Possible esponse variables
response_variables = ['is_recid', 'is_violent_recid', 'two_year_recid']

# Variables that are used for data cleaning
check_variables = ['days_b_screening_arrest']

# Subselect data
df = raw_data[all_features+response_variables+check_variables]

# Apply filters
df = df[(df['days_b_screening_arrest'] <= 30) & 
        (df['days_b_screening_arrest'] >= -30) & 
        (df['is_recid'] != -1) & 
        (df['c_charge_degree'] != 'O')]

df = df[all_features+response_variables]
print('Dataframe has {} rows and {} columns.'.format(df.shape[0], df.shape[1]))

#Variables like charge degree and prior offenses count seem like they might be could predictors. 
#Variables relating to the recidivism offense, like recidivism score, seem like sensible outcome variables.

#1.3
# Code binary features as 0 and 1
for x in binary_categorical_features:
    for new_value, value in enumerate(set(df[x])):
        print("Replace {} with {}.".format(value, new_value))
        df = df.replace(value, new_value)

# Use 1-hot encoding for other categorical variables
one_hot_features = []
for x in other_categorical_features:
    for new_feature, value in enumerate(set(df[x])):
        feature_name = "{}_is_{}".format(x,value)
        df.insert(3, feature_name, df[x]==value)
        one_hot_features += [feature_name]

# Check what the data frame looks like now
df.head(10)

#1.4
# list of features
features = numerical_features + binary_categorical_features + one_hot_features

# features data frame
X = df[features]

# responses data frame
Y = df[response_variables]

# Split the data into a training set containing 90% of the data
# and test set containing 10% of the data

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

#2.1
# Create a model
tree = DecisionTreeClassifier(random_state=42)

# Fit model to training data
tree.fit(X_train, Y_train)

# Evaluate training accuracy
train_predictions = tree.predict(X_train)
accuracy = accuracy_score(Y_train, train_predictions)

# Check size of decision tree
num_leaves = tree.get_n_leaves()

# Report results
print('Trained decision tree with {} leaves and training accuracy {:.2f}.'.format(num_leaves, accuracy))

#2.2
# Perform 5-fold cross-validation for different tree sizes

print('Leaves\tMean accuracy')
print('---------------------')
# With 100-1800 test, highest scores were 100 and 200, so size is adjusted accordingly
for num_leaves in range(2,250,2):

    # Trees must have at least 2 leaves
    if num_leaves >= 2:

        # construct a classifier with a limit on its number of leaves
        temptree = DecisionTreeClassifier(max_leaf_nodes=num_leaves, random_state=42)
        
        # Get validation accuracy via 5-fold cross-validation
        scores = cross_val_score(temptree, X_train, Y_train, cv=5, scoring='accuracy')

    print("{}\t{:.5f}".format(num_leaves,scores.mean()))

#2.3 
# In the previous test, we saw that 48 leaves yielded optimal accuracy
# Create a model with the best number of leaves
besttree = DecisionTreeClassifier(max_leaf_nodes=48, random_state=42)

# Fit model to training data
besttree.fit(X_train, Y_train)

# Evaluate training accuracy
train_predictions = besttree.predict(X_train)
train_accuracy = accuracy_score(Y_train, train_predictions)

# Evaluate test accuracy
test_predictions = besttree.predict(X_test)
test_accuracy = accuracy_score(Y_test, test_predictions)

# Check size of decision tree
num_leaves = besttree.get_n_leaves()

# Report results
print('Trained decision tree with {} leaves has training accuracy {:.2f}.'.format(num_leaves, train_accuracy))
print('Trained decision tree with {} leaves has test accuracy {:.2f}.'.format(num_leaves, test_accuracy))

#3.2
# Create subset of training data without information on race. 
# (The information on race was encoded in the one-hot features.)
remaining_features = [v for v in X.columns if v not in one_hot_features]
X_train_sub = X_train[remaining_features]
X_test_sub = X_test[remaining_features]

# Create a model
dtc = DecisionTreeClassifier(max_leaf_nodes=39)
    
# Fit model to training data
dtc.fit(X_train_sub, Y_train['two_year_recid'])

# Evaluate training accuracy
y_pred = dtc.predict(X_test_sub)
accuracy = (y_pred == Y_test['two_year_recid']).mean()

# Check size of decision tree
num_leaves = dtc.get_n_leaves()

# Report results
print('Trained decision tree with {} leaves and test accuracy {:.2f}.'.format(num_leaves, accuracy))

#There is a 10-12% boost in accuracy by removing race as a feature in this classification problem. 
#This implies that race is a bad predictor and introduces unnecessary noise to our model that disappears when race is removed as a feature

#3.3
tree_without_race = export_text(dtc, feature_names=list(X_train_sub.columns))
print("tree without race:")
print(tree_without_race)
tree_with_race = export_text(besttree, feature_names=list(X_train.columns))
print("tree with race:")
print(tree_with_race)

#There are several locations in the first decision tree with race that include decisions that are made based on the race of the individual.
#The fact that decisions are made based on the race of the invididual indicate racial bias.

#4.3
# Fit LDA
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, Y_train['two_year_recid'])
lda_pred = lda.predict(X_test)
lda_accuracy = accuracy_score(Y_test['two_year_recid'], lda_pred)
print(f'LDA test accuracy: {lda_accuracy:.2f}')

# Fit Logistic Regression
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, Y_train['two_year_recid'])
log_reg_pred = log_reg.predict(X_test)
log_reg_accuracy = accuracy_score(Y_test['two_year_recid'], log_reg_pred)
print(f'Logistic Regression test accuracy: {log_reg_accuracy:.2f}')

# Random Forest
rf = RandomForestClassifier()
param_dist_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_search = RandomizedSearchCV(rf, param_dist_rf, n_iter=10, cv=5, random_state=42)
rf_search.fit(X_train, Y_train['two_year_recid'])
rf_best = rf_search.best_estimator_
rf_pred = rf_best.predict(X_test)
rf_accuracy = accuracy_score(Y_test['two_year_recid'], rf_pred)
print(f'Random Forest test accuracy: {rf_accuracy:.2f}')

# Gradient Boosting
gb = GradientBoostingClassifier()
param_dist_gb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

gb_search = RandomizedSearchCV(gb, param_dist_gb, n_iter=10, cv=5, random_state=42)
gb_search.fit(X_train, Y_train['two_year_recid'])
gb_best = gb_search.best_estimator_
gb_pred = gb_best.predict(X_test)
gb_accuracy = accuracy_score(Y_test['two_year_recid'], gb_pred)
print(f'Gradient Boosting test accuracy: {gb_accuracy:.2f}')

#Bagging Classifier
bagging = BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=100, random_state=42)
bagging.fit(X_train, Y_train['two_year_recid'])
bagging_pred = bagging.predict(X_test)
bagging_accuracy = accuracy_score(Y_test['two_year_recid'], bagging_pred)
print(f'Bagging Classifier test accuracy: {bagging_accuracy:.2f}')

# Tune and fit SVC
svc = SVC()
param_dist_svc = {
    'C': [0.1, 1, 10],
    'gamma': [0.1, 0.01],
    'kernel': ['rbf']
}

svc_search = RandomizedSearchCV(svc, param_dist_svc, n_iter=5, cv=3, random_state=42)
svc_search.fit(X_train, Y_train['two_year_recid'])
svc_best = svc_search.best_estimator_
svc_pred = svc_best.predict(X_test)
svc_accuracy = accuracy_score(Y_test['two_year_recid'], svc_pred)
print(f'SVC test accuracy: {svc_accuracy:.2f}')

#All the models have comprable accuracy (within 2%) to the race-free tree, but only the bagging does worse.


Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')
Dataframe has 6172 rows and 11 columns.
Replace F

/var/folders/wm/749y2zb56vj71_nxgcgvxdw80000gn/T/ipykernel_41047/2294658904.py:57: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(value, new_value)


16	0.57004
18	0.57094
20	0.56878
22	0.57832
24	0.57778
26	0.57796
28	0.58354
30	0.58246
32	0.58228
34	0.58192
36	0.58084
38	0.57976
40	0.58606
42	0.58498
44	0.58624
46	0.58642
48	0.58642
50	0.58606
52	0.58570
54	0.58570
56	0.58570
58	0.58516
60	0.58426
62	0.58426
64	0.58426
66	0.58372
68	0.58246
70	0.58120
72	0.57994
74	0.58012
76	0.57652
78	0.57616
80	0.57940
82	0.57742
84	0.57670
86	0.57616
88	0.57598
90	0.57562
92	0.57706
94	0.57688
96	0.57508
98	0.57490
100	0.57364
102	0.57328
104	0.57364
106	0.57382
108	0.57346
110	0.57328
112	0.57652
114	0.57526
116	0.57454
118	0.57364
120	0.57346
122	0.57364
124	0.57346
126	0.57346
128	0.57256
130	0.57238
132	0.57238
134	0.57256
136	0.57166
138	0.57274
140	0.57256
142	0.57274
144	0.57292
146	0.57202
148	0.57274
150	0.57148
152	0.57094
154	0.57040
156	0.57004
158	0.56986
160	0.56968
162	0.56932
164	0.56860
166	0.56878
168	0.56878
170	0.56716
172	0.56716
174	0.56770
176	0.56698
178	0.56698
180	0.56788
182	0.56752
184	0.56770
186	0.56752
188	0.5669